In [13]:
class neuralNetwork:
  def __init__(self, input, hidden, output, learning_rate):
    self.input = input
    self.hidden = hidden
    self.output = output

    self.lr = learning_rate
    pass

  def train():
    pass

  def query():
    pass

In [14]:
# layers
input_nodes = 3
hidden_nodes = 3
output_nodes = 3

# set learning rate
lr = 0.3

# instantiate the NN
n = neuralNetwork(input_nodes, hidden_nodes, output_nodes, lr)